# Introduction
Many people are lured to the life in a big city. The promises of a city that never sleeps, with more nationalities than you can count, and the possibities to eat food from every region in the world is a massive attraction. But.. What is the difference really, between a city like Amsterdam and one like New York? Is there really such a huge difference ? For this exercise we'll explore the culinary options in both cities to figure out if making the jump overseas will make a foodies heart jump with joy. We will dive into the neighbourhoods to figure out how many different kitchens we can find and how many of them there are.

We will be using the Foursquare API to get information about the venues. For New York we will rely on the "newyork_data.json" data file which contains the boroughs, the neighbourhoods and their location. For Amsterdam, we will use the data from https://allecijfers.nl/gemeente-overzicht/amsterdam/ to find out about all the neighbourhoods. We'll grab the location data from the Geocoder Python package